Part One - Retreiving and Analysing mRNA and Protein Sequences of Human Cadherin-7 (25 marks)


- Task 1 - Search NCBI for the human Cadherin-7 gene entry, how many alternate transcripts does it have, what are their official accession ids, and how long are they in nucleotides (5 marks)

- Task 2 - Calculate the percentage composition of A, C, G, and T for each transcript and report the results in a table with the transcripts as rows and the percentages for each nucleotide as columns (5 marks)

- Task 3 - Translate the transcript sequences into protein sequences, report the length of the resulting proteins and state the most frequent amino acid in each one, if there are joint frequent ones name them all (5 marks)


- up to 6 marks for including one additional piece of analysis.

- up to 4 marks for exceptionally well organised and executed approach.


Possible extensions here might be to:

- look at other human Cadherin genes to see whether they also use alternate splicing to create different isoforms

- compare human Cadherin-7 to orthologs in other species

- write a short summary of what Cadherins do

- look at the structure of the Cadherin protein family

- comment on the difference in structure between the different Cadherin-7 isoforms

# Task 1

not gonna write code for this one I don't think!

Searching up "Homo Sapiens Cadherin-7" in Genes database, clicking through to "NCBI reference sequences" then tryed the new "Transcript table" which was super clear and helpful as it already included the length in nucleotides!

4 alternate transcripts (accession number : length (nt)):
- NM_004361.5 : 12136
- NM_033646.4 : 12126
- NM_001317214.3 : 3407
- NM_001262438.2 : 12938

In [ ]:
# TASK 2

